In [6]:
import easyocr
import matplotlib.pyplot as plt
import cv2 as cv
from ultralytics import YOLO
import time
import numpy as np
import PIL



In [7]:
def matchlabel(label):
    product_dict={'foxin wired mouse':['foxin','wired','mouse'],'vaseline bluesea':['vaseline','bluesea'], 'close up':['close','up'],'set wet deodorant':['set','wet','deodorant'],  'axe deodorant':['axe','deodorant'],  'wisdom tooth brush':['wisdom','tooth','brush'],  'Nivea body cream':['Nivea','body','cream'],  'peel off mark':['peel' ,'off' ,'mark'],  'hit fresh':['hit' ,'fresh'],  'bee black shoe polish':['bee' ,'black', 'shoe' ,'polish'],  'dettol cool':['dettol' ,'cool'],  'active wheel':['active' ,'wheel'],  'surf excel easy wash':['surf' ,'excel' ,'easy' ,'wash'],  'home ninja hand wash':['home' ,'ninja', 'hand', 'wash'],'bisleri water':['bisleri', 'water'],'soyhurt':['soyhurt'],'barvo orange juice':['barvo', 'orange' ,'juice',],'good morgan apple juice':['good', 'morgan' ,'apple' ,'juice',],'vanilla milk':['vanilla', 'milk'],'other':[]}
    for word in label.split():
        for key in list(product_dict.keys()):
            
            if word in product_dict[key]:
                return key
    return 

def findname(text):
    label=""
    bank = ['foxin','wired','mouse','vaseline','bluesea','close','up','set','wet','deodorant','axe','deodorant','wisdom','tooth','brush','Nivea','body','cream', 'peel' ,'off' ,'mark',  'hit' ,'fresh',  'bee' ,'black', 'shoe' ,'polish',  'dettol' ,'cool',  'active' ,'wheel',  'surf' ,'excel' ,'easy' ,'wash',  'home' ,'ninja', 'hand', 'wash','bisleri', 'water','soyhurt','barvo', 'orange' ,'juice','good', 'morgan' ,'apple' ,'juice','vanilla', 'milk']
    textlist=text.lower().split()
    for text in textlist:
        if text in bank:
            label+=text+" "
    return label

In [9]:
def get_label(frame,reader):
    gray = cv.cvtColor(frame,cv.COLOR_RGB2GRAY)
    result = reader.readtext(gray)
    text=""

    for res in result:
        text += res[1] + " "

    label=findname(text)
    text = matchlabel(label)
    return text

In [15]:
def run_yolo(frame,model,inventory):
    class_names = ['fresh apple', 'fresh banana', 'fresh bellpepper', 'fresh carrot', 'fresh cucumber', 'fresh mango', 
               'fresh orange', 'fresh potato', 'rotten apple', 'rotten banana', 'rotten carrot', 
               'rotten cucumber', 'rotten mango', 'rotten orange', 'rotten potato', 'rotten tomato', 'rottenbellpepper']
    results = model(frame)

    # Extracting detection results
    fruits=[]
    for result in results:
        boxes = result.boxes  # Boxes: [xyxy, conf, cls]
        for box in boxes:
            # Extract coordinates and class index
            if box.conf[0]>=0.6:
                x1, y1, x2, y2 = map(int, box.xyxy[0])  # Bounding box coordinates
                conf = box.conf[0]  # Confidence score
                class_id = int(box.cls[0])  # Class ID

                # Get class name
                class_name = class_names[class_id]
                if class_name is not None:
                    if class_name not in list(inventory.keys()):
                        inventory[class_name]=1
                    else:
                        inventory[class_name]+=1
            
                # fruits.append(class_name)
                cv.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Green box
                cv.putText(frame, f"{class_name} {conf:.2f}", (x1, y1 - 10), cv.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
                


In [19]:
def get_date(frame,model,reader):
    
    results = model(frame)
    
    

    # Extracting detection results
    fruits=[]
    for result in results:
        boxes = result.boxes  # Boxes: [xyxy, conf, cls]
        for box in boxes:
            # Extract coordinates and class index
            if box.conf[0]>=0.6:
                x1, y1, x2, y2 = map(int, box.xyxy[0])  # Bounding box coordinates
                conf = box.conf[0]  # Confidence score
                class_id = int(box.cls[0])  # Class ID
                roi = frame[y1:y2,x1:x2]
                gray = cv.cvtColor(roi,cv.COLOR_RGB2GRAY)
                dateframe = reader.readtext(gray)
                cv.putText(frame, f"Exp Date: {dateframe}", (x1, y1 - 10), cv.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

                
            
                
                cv.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Green box
                
                


In [22]:
model = YOLO("model/best.pt")
ocrmodel = YOLO("model/date.pt")

inventory = {}
reader = easyocr.Reader(['en'],gpu=True)
vid = cv.VideoCapture(0)
# img = cv.imread('/home/david/OCR/easyocr/detection_ws/data/images/train/20241012_120532.jpg')
skip_frame = True
while True:
    try:
        a=time.time()
        ret, frame = vid.read()
        if not ret:
            print("Failed to grab frame")
            break
        
        text=get_label(frame=frame,reader=reader)
        if text is not None:
            product_name = 'Product Name: '+text
            cv.putText(frame,product_name,(50,70), cv.FONT_HERSHEY_SIMPLEX,1,(50,50,255),2)
            cv.imshow('Camera', frame)
            if text not in list(inventory.keys()):
                inventory[text]=1
            else:
                inventory[text]+=1
            get_date(frame,ocrmodel,reader)
            cv.putText(frame,f"Qty: {inventory[text]}",(60,70), cv.FONT_HERSHEY_SIMPLEX,1,(50,50,255),2)
            time.sleep(0.01)

        else:
            fruits=run_yolo(frame,model)
            time.sleep(0.01)
            
            cv.imshow('Camera', frame)
    

        
    
        # Press 'q' to exit the loop
        if cv.waitKey(1) == ord('q'):
            break
    
        # print(fps)
        print(text)
        
    except Exception as e:
        print(f"Error: {e}")
        break

# Release the capture and writer objects
vid.release()
cv.destroyAllWindows()


0: 480x640 1 fresh potato, 25.3ms
Speed: 2.7ms preprocess, 25.3ms inference, 12.3ms postprocess per image at shape (1, 3, 480, 640)
None


QObject::moveToThread: Current thread (0x20e6d280) is not the object's thread (0x221aab60).
Cannot move to target thread (0x20e6d280)

QObject::moveToThread: Current thread (0x20e6d280) is not the object's thread (0x221aab60).
Cannot move to target thread (0x20e6d280)

QObject::moveToThread: Current thread (0x20e6d280) is not the object's thread (0x221aab60).
Cannot move to target thread (0x20e6d280)

QObject::moveToThread: Current thread (0x20e6d280) is not the object's thread (0x221aab60).
Cannot move to target thread (0x20e6d280)

QObject::moveToThread: Current thread (0x20e6d280) is not the object's thread (0x221aab60).
Cannot move to target thread (0x20e6d280)

QObject::moveToThread: Current thread (0x20e6d280) is not the object's thread (0x221aab60).
Cannot move to target thread (0x20e6d280)

QObject::moveToThread: Current thread (0x20e6d280) is not the object's thread (0x221aab60).
Cannot move to target thread (0x20e6d280)

QObject::moveToThread: Current thread (0x20e6d280) is n


0: 480x640 1 fresh potato, 1 rotten banana, 22.2ms
Speed: 3.2ms preprocess, 22.2ms inference, 4.5ms postprocess per image at shape (1, 3, 480, 640)
None

0: 480x640 1 fresh potato, 21.4ms
Speed: 4.8ms preprocess, 21.4ms inference, 12.4ms postprocess per image at shape (1, 3, 480, 640)
None

0: 480x640 1 fresh potato, 1 rotten banana, 14.1ms
Speed: 3.8ms preprocess, 14.1ms inference, 4.0ms postprocess per image at shape (1, 3, 480, 640)
None

0: 480x640 1 fresh potato, 1 rotten banana, 21.1ms
Speed: 2.5ms preprocess, 21.1ms inference, 12.4ms postprocess per image at shape (1, 3, 480, 640)
None

0: 480x640 1 fresh potato, 17.9ms
Speed: 2.8ms preprocess, 17.9ms inference, 3.5ms postprocess per image at shape (1, 3, 480, 640)
None

0: 480x640 1 fresh potato, 21.2ms
Speed: 4.3ms preprocess, 21.2ms inference, 12.5ms postprocess per image at shape (1, 3, 480, 640)
None

0: 480x640 1 fresh potato, 22.3ms
Speed: 4.9ms preprocess, 22.3ms inference, 12.6ms postprocess per image at shape (1, 3, 4